# Evaluating Robustness
This tutorial walks through how to use `Augmenty`/`SpaCy` augmenters to evalutate robustness of any NLP pipeline. As an example we'll start out by evaluating SpaCy small and DaCy small on the test set of [DaNE](https://github.com/alexandrainst/danlp/blob/master/docs/docs/datasets.md#dane). DaNE is the Danish Dependency treebank tagged for part-of-speech tags, dependency relations and named entities. Lastly we will show how to use this framework on any other type of model using [DaNLP's BERT](https://github.com/alexandrainst/danlp/blob/master/docs/docs/tasks/ner.md#-bert-bert) as an example. 

Let us start of with installing the required packages and loading the models and dataset we wish to test on.

### Installing packages

To get started we will first need to install a few packages:

```bash
# install models
pip install dacy
python -m spacy download da_core_news_sm

# install augmentation library
pip install "augmenty>1.3.0"
```

## Loading models and data

In [7]:
from dacy.datasets import dane

# load the DaNE test set
test = dane(splits=["test"])

/Users/au561649/Desktop/Github/DaCy/.venv/lib/python3.9/site-packages/spacy/util.py:833: UserWarning: [W095] Model 'da_dacy_medium_trf' (0.1.0) was trained with spaCy v3.1 and may not be 100% compatible with the current version (3.2.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/Users/au561649/Desktop/Github/DaCy/.venv/lib/python3.9/site-packages/spacy/util.py:833: UserWarning: [W095] Model 'da_dacy_small_trf' (0.1.0) was trained with spaCy v3.1 and may not be 100% compatible with the current version (3.2.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/Users/au561649/Desktop/Github/DaCy/.venv/lib/python3.9/sit

In [ ]:
import spacy
import dacy

# load models
spacy_small = spacy.load("da_core_news_sm")
dacy_small = dacy.load("small")

## Estimating performance
Evaluating models already in the `SpaCy` framework is very straightforward. Simply call the `score` function on your nlp pipeline and choose which metrics you want to calculate performance for. `score` is a wrapper for `SpaCy.scorer.Scorer` that outputs a nicely formatted dataframe. `score` calculates performance for NER, POS, tokenization, and dependency parsing by default, which can be changed with the score_fn argument.

In [8]:
from dacy.score import score

spacy_baseline = score(test, apply_fn=spacy_small, score_fn=["ents", "pos"])
dacy_baseline = score(test, apply_fn=dacy_small, score_fn=["ents", "pos"])

/Users/au561649/Desktop/Github/DaCy/.venv/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/Users/au561649/Desktop/Github/DaCy/.venv/lib/python3.9/site-packages/spacy/pipeline/attributeruler.py:150: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  matches = self.matcher(doc, allow_missing=True, as_spans=False)


In [9]:
spacy_baseline

wall_time    ents_p    ents_r    ents_f  ents_per_type_MISC_p  \
0   1.638683  0.720408  0.632616  0.673664              0.649485   

   ents_per_type_MISC_r  ents_per_type_MISC_f  ents_per_type_LOC_p  \
0              0.520661              0.577982             0.653846   

   ents_per_type_LOC_r  ents_per_type_LOC_f  ...  ents_per_type_ORG_f  \
0             0.708333                 0.68  ...             0.551724   

   ents_per_type_PER_p  ents_per_type_PER_r  ents_per_type_PER_f  \
0             0.793651             0.833333             0.813008   

   ents_excl_MISC_ents_p  ents_excl_MISC_ents_r  ents_excl_MISC_ents_f  \
0               0.737913               0.663616               0.698795   

    pos_acc   tag_acc  k  
0  0.949103  0.949103  0  

[1 rows x 22 columns]

In [10]:
dacy_baseline

wall_time    ents_p    ents_r    ents_f  ents_per_type_MISC_p  \
0  12.794494  0.774312  0.756272  0.765186                 0.656   

   ents_per_type_MISC_r  ents_per_type_MISC_f  ents_per_type_LOC_p  \
0              0.677686              0.666667             0.736364   

   ents_per_type_LOC_r  ents_per_type_LOC_f  ...  ents_per_type_PER_f  \
0              0.84375             0.786408  ...              0.90027   

   ents_per_type_ORG_p  ents_per_type_ORG_r  ents_per_type_ORG_f  \
0             0.773109             0.571429             0.657143   

   ents_excl_MISC_ents_p  ents_excl_MISC_ents_r  ents_excl_MISC_ents_f  \
0               0.809524               0.778032               0.793466   

   pos_acc  tag_acc  k  
0  0.98002      0.0  0  

[1 rows x 22 columns]

### Estimating robustness and biases
To obtain performance estimates on augmented data, simply provide a list of augmenters as the `augmenters` argument. 

In [11]:
from augmenty.span.entities import create_per_replace_augmenter_v1
from dacy.datasets import female_names
from spacy.training.augment import create_lower_casing_augmenter

In [12]:

lower_aug = create_lower_casing_augmenter(level=1)
female_name_dict = female_names()
# Augmenter that replaces names with random Danish female names. Keep the format of the name as is (force_pattern_size=False)
# but replace the name with one of the two defined patterns

patterns = [["firstname"], ["firstname", "lastname"], ["firstname", "firstname", "lastname"]]
female_aug = create_per_replace_augmenter_v1(female_name_dict, patterns, level=0.1)

spacy_aug = score(
    test,
    apply_fn=spacy_small,
    score_fn=["ents", "pos"],
    augmenters=[lower_aug, female_aug],
)
dacy_aug = score(
    test,
    apply_fn=dacy_small,
    score_fn=["ents", "pos"],
    augmenters=[lower_aug, female_aug],
)

/Users/au561649/Desktop/Github/DaCy/.venv/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/Users/au561649/Desktop/Github/DaCy/.venv/lib/python3.9/site-packages/spacy/pipeline/attributeruler.py:150: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  matches = self.matcher(doc, allow_missing=True, as_spans=False)


In [13]:
import pandas as pd

pd.concat([spacy_baseline, spacy_aug])

wall_time    ents_p    ents_r    ents_f  ents_per_type_MISC_p  \
0   1.638683  0.720408  0.632616  0.673664              0.649485   
0   1.679577  0.673267  0.243728  0.357895              0.741935   
0   1.375843  0.720408  0.632616  0.673664              0.649485   

   ents_per_type_MISC_r  ents_per_type_MISC_f  ents_per_type_LOC_p  \
0              0.520661              0.577982             0.653846   
0              0.380165              0.502732             0.653846   
0              0.520661              0.577982             0.653846   

   ents_per_type_LOC_r  ents_per_type_LOC_f  ...  ents_per_type_ORG_f  \
0             0.708333             0.680000  ...             0.551724   
0             0.354167             0.459459  ...             0.153846   
0             0.708333             0.680000  ...             0.551724   

   ents_per_type_PER_p  ents_per_type_PER_r  ents_per_type_PER_f  \
0             0.793651             0.833333             0.813008   
0             0.626866             0.233333             0.340081   
0             0.793651             0.833333             0.813008   

   ents_excl_MISC_ents_p  ents_excl_MISC_ents_r  ents_excl_MISC_ents_f  \
0               0.737913               0.663616               0.698795   
0               0.642857               0.205950               0.311958   
0               0.737913               0.663616               0.698795   

    pos_acc   tag_acc  k  
0  0.949103  0.949103  0  
0  0.920288  0.920288  0  
0  0.949103  0.949103  0  

[3 rows x 22 columns]

In [14]:
pd.concat([dacy_baseline, dacy_aug])

wall_time    ents_p    ents_r    ents_f  ents_per_type_MISC_p  \
0  12.794494  0.774312  0.756272  0.765186              0.656000   
0  13.063498  0.727088  0.639785  0.680648              0.614754   
0  12.604465  0.774312  0.756272  0.765186              0.656000   

   ents_per_type_MISC_r  ents_per_type_MISC_f  ents_per_type_LOC_p  \
0              0.677686              0.666667             0.736364   
0              0.619835              0.617284             0.714286   
0              0.677686              0.666667             0.736364   

   ents_per_type_LOC_r  ents_per_type_LOC_f  ...  ents_per_type_PER_f  \
0              0.84375             0.786408  ...             0.900270   
0              0.78125             0.746269  ...             0.805797   
0              0.84375             0.786408  ...             0.900270   

   ents_per_type_ORG_p  ents_per_type_ORG_r  ents_per_type_ORG_f  \
0             0.773109             0.571429             0.657143   
0             0.686869             0.422360             0.523077   
0             0.773109             0.571429             0.657143   

   ents_excl_MISC_ents_p  ents_excl_MISC_ents_r  ents_excl_MISC_ents_f  \
0               0.809524               0.778032               0.793466   
0               0.764228               0.645309               0.699752   
0               0.809524               0.778032               0.793466   

    pos_acc  tag_acc  k  
0  0.980020      0.0  0  
0  0.974477      0.0  0  
0  0.980020      0.0  0  

[3 rows x 22 columns]

In the second row, we see that `SpaCy small` is very vulnerable to lower casing as NER recall drops from 0.66 to 0.38. `DaCy small` is slightly more robust lower casing, but still suffers. Changing names also leads to a drop in performance for both models. 

To better estimate the effect of stochastic augmenters such as those changing names or adding keystroke errors we can use the `k` argument in `score` to run the augmenter multiple times.

In [18]:
from augmenty.character.replace import create_keystroke_error_augmenter_v1

key_05_aug = create_keystroke_error_augmenter_v1(level=0.5, keyboard="da_qwerty_v1")

spacy_key = score(
    test, apply_fn=spacy_small, score_fn=["ents", "pos"], augmenters=[key_05_aug], k=5
)

In [19]:
spacy_key

wall_time    ents_p    ents_r    ents_f  ents_per_type_MISC_p  \
0   4.706608  0.110333  0.112903  0.111603              0.040000   
1   3.668705  0.118068  0.118280  0.118174              0.080000   
2   4.472808  0.086342  0.098566  0.092050              0.038217   
3   5.064881  0.119816  0.139785  0.129032              0.060403   
4   4.583604  0.116239  0.121864  0.118985              0.051471   

   ents_per_type_MISC_r  ents_per_type_MISC_f  ents_per_type_LOC_p  \
0              0.041322              0.040650             0.101852   
1              0.082645              0.081301             0.127451   
2              0.049587              0.043165             0.099099   
3              0.074380              0.066667             0.165289   
4              0.057851              0.054475             0.085714   

   ents_per_type_LOC_r  ents_per_type_LOC_f  ...  ents_per_type_ORG_f  \
0             0.114583             0.107843  ...             0.089655   
1             0.135417             0.131313  ...             0.089744   
2             0.114583             0.106280  ...             0.079755   
3             0.208333             0.184332  ...             0.101266   
4             0.093750             0.089552  ...             0.100334   

   ents_per_type_PER_p  ents_per_type_PER_r  ents_per_type_PER_f  \
0             0.162679             0.188889             0.174807   
1             0.160221             0.161111             0.160665   
2             0.122549             0.138889             0.130208   
3             0.146018             0.183333             0.162562   
4             0.179612             0.205556             0.191710   

   ents_excl_MISC_ents_p  ents_excl_MISC_ents_r  ents_excl_MISC_ents_f  \
0               0.130045               0.132723               0.131370   
1               0.129032               0.128146               0.128588   
2               0.102083               0.112128               0.106870   
3               0.137450               0.157895               0.146965   
4               0.135857               0.139588               0.137698   

    pos_acc   tag_acc  k  
0  0.331013  0.331013  0  
1  0.329741  0.329741  1  
2  0.326347  0.326347  2  
3  0.336281  0.336281  3  
4  0.330561  0.330561  4  

[5 rows x 22 columns]

In this manner, evaluating performance on augmented data for SpaCy pipelines is as easy as defining the augmenters and calling a single function. In the `dacy_paper_replication.py` script you can find the exact script used to evaluate the robustness of Danish NLP models in the [DaCy paper]().
